In [4]:
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import f_oneway
import numpy as np

# Accuracies from different classifiers for 7 participants (Fat-a, Fat-b, Fat-c, Fat-d etc)
classifier_accuracies = {
    'LDA': [90, 92.5, 97.5, 95, 95, 85, 95], 
    'SVM(linear Default)': [88, 93, 97, 95, 95, 90, 97],  
    'SVM(rbf custom)': [90, 92.5, 97.5, 95, 97.5, 90, 90],  
    'Logistic Regression': [87.5, 90, 97.5, 95, 97.5, 92.5, 97.5],  
    'Complex Tree': [87.5, 87.5, 90.5, 92.5, 90, 80, 87.5],  
    'Naive Bayes': [90, 92.5, 97.5, 95, 95, 87.5, 92.5],  
    'KNN': [90, 85, 97.5, 95, 97.5, 90, 92.5]  
}

# Flatten data for Tukey test
data = pd.DataFrame(classifier_accuracies).melt(var_name='Classifier', value_name='Accuracy')

# Perform one-way ANOVA
f_value, p_value = f_oneway(*classifier_accuracies.values())
print(f"ANOVA F-value: {f_value:.4f}, p-value: {p_value:.4f}")

# Perform Tukey HSD post-hoc test
tukey = pairwise_tukeyhsd(endog=data['Accuracy'], groups=data['Classifier'], alpha=0.05)
print(tukey.summary())

# Calculate mean and standard deviation
mean_accuracies = {classifier: np.mean(accuracies) for classifier, accuracies in classifier_accuracies.items()}
std_accuracies = {classifier: np.std(accuracies) for classifier, accuracies in classifier_accuracies.items()}

# Print mean and standard deviation
print("\nMean and Standard Deviation of Accuracies:")
for classifier in classifier_accuracies.keys():
    print(f"{classifier}: Mean accuracy = {mean_accuracies[classifier]:.2f}%, Standard Deviation = {std_accuracies[classifier]:.2f}%")


ANOVA F-value: 1.9138, p-value: 0.1009
             Multiple Comparison of Means - Tukey HSD, FWER=0.05              
       group1              group2       meandiff p-adj   lower   upper  reject
------------------------------------------------------------------------------
       Complex Tree                 KNN   4.5714 0.3173 -1.8613 11.0041  False
       Complex Tree                 LDA   4.9286 0.2359 -1.5041 11.3613  False
       Complex Tree Logistic Regression      6.0 0.0817 -0.4327 12.4327  False
       Complex Tree         Naive Bayes   4.9286 0.2359 -1.5041 11.3613  False
       Complex Tree SVM(linear Default)   5.6429 0.1194 -0.7898 12.0755  False
       Complex Tree     SVM(rbf custom)   5.2857 0.1702  -1.147 11.7184  False
                KNN                 LDA   0.3571    1.0 -6.0755  6.7898  False
                KNN Logistic Regression   1.4286 0.9926 -5.0041  7.8613  False
                KNN         Naive Bayes   0.3571    1.0 -6.0755  6.7898  False
             

In [2]:
from scipy.stats import ttest_rel
import itertools

# Extract classifier names
classifier_names = list(classifier_accuracies.keys())

# Iterate over all pairs of classifiers and perform T-tests
t_test_results = {}
for (clf1, clf2) in itertools.combinations(classifier_names, 2):
    acc1 = classifier_accuracies[clf1]
    acc2 = classifier_accuracies[clf2]
    
    t_stat, p_value = ttest_rel(acc1, acc2)
    t_test_results[(clf1, clf2)] = (t_stat, p_value)

# Print T-test results
for key, value in t_test_results.items():
    print(f"T-test between {key[0]} and {key[1]}: t-value = {value[0]:.4f}, p-value = {value[1]:.4f}")


T-test between LDA and SVM(linear Default): t-value = -0.8462, p-value = 0.4299
T-test between LDA and SVM(rbf custom): t-value = -0.3111, p-value = 0.7663
T-test between LDA and Logistic Regression: t-value = -0.8115, p-value = 0.4481
T-test between LDA and Complex Tree: t-value = 6.7019, p-value = 0.0005
T-test between LDA and Naive Bayes: t-value = 0.0000, p-value = 1.0000
T-test between LDA and KNN: t-value = 0.2402, p-value = 0.8182
T-test between SVM(linear Default) and SVM(rbf custom): t-value = 0.3017, p-value = 0.7731
T-test between SVM(linear Default) and Logistic Regression: t-value = -0.5008, p-value = 0.6343
T-test between SVM(linear Default) and Complex Tree: t-value = 4.3292, p-value = 0.0049
T-test between SVM(linear Default) and Naive Bayes: t-value = 0.8839, p-value = 0.4108
T-test between SVM(linear Default) and KNN: t-value = 0.7457, p-value = 0.4840
T-test between SVM(rbf custom) and Logistic Regression: t-value = -0.5477, p-value = 0.6036
T-test between SVM(rbf cu

In [11]:
from scipy.stats import binom_test

binomial_test_results = {}

for clf in classifier_names:
    accuracies = classifier_accuracies[clf]
    
    # Convert accuracies to success counts (e.g., above 50% is a success)
    successes = sum([1 for acc in accuracies if acc > 50])
    n = len(accuracies)  # Number of trials
    
    # Perform binomial test against a baseline of 50%
    p_value = binom_test(successes, n, 0.5, alternative='greater')
    
    binomial_test_results[clf] = p_value

# Print Binomial test results
for clf, p_value in binomial_test_results.items():
    print(f"Binomial test for {clf}: Success={successes}, p-value = {p_value:.4f}")


Binomial test for LDA: Success=7, p-value = 0.0078
Binomial test for SVM(linear Default): Success=7, p-value = 0.0078
Binomial test for SVM(rbf custom): Success=7, p-value = 0.0078
Binomial test for Logistic Regression: Success=7, p-value = 0.0078
Binomial test for Complex Tree: Success=7, p-value = 0.0078
Binomial test for Naive Bayes: Success=7, p-value = 0.0078
Binomial test for KNN: Success=7, p-value = 0.0078


/var/folders/rw/tyywsmk172746357n7yyqb500000gn/T/ipykernel_47370/721277079.py:13: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_value = binom_test(successes, n, 0.5, alternative='greater')
